# p5.js in the Jupyter Notebook for custom interactive visualizations 


The goal of this notebook is to show how to create custom visualizations in a Jupyter Notebook.

The Jupyter Notebook provides the `%%javascript` cell magic, that we can use to load existing Javascript libraries.

Although this can be applied in many contexts and applications, we will focus on one concrete example.


## Example with the Bermuda Triangle Puzzle


The Bermuda Triangle Puzzle is a wooden puzzle consisting of 1 big triangle and 16 smaller triangles of equal size.

On the edges of the big triangle, there are colored dots, as well as on the edges of the small triangles:

![Bermuda Triangle Puzzle](./bermuda_triangle_puzzle.jpg)

## Goal

The 16 triangles all fit inside the big triangle. But the goal of the puzzle is to **arrange the triangles in a way so that each adjacent color is the same: red next to red, blue next to blue, and so on...**.

One thing to notice is that the colors on the edges of the big triangle cannot be moved, but the smaller triangles can be rotated and re-arranged as we want.

### How do we solve this?

As usual, there are different approaches:

1. One can just take the puzzle and start playing with it. After a while, some patterns may show upbut can become very tedious when trying all combinations.
2. Speaking of trying all combinations, one can try to brute force the puzzle (still manually with the wooden pieces), take notes of the configurations that have been tested, and keep iterating. This requires a lot of rigor to not miss any step and start from the beginning.
3. Brute-forcing can be fun for a moment, but repetitive after a few minutes. Instead of doing it manually, we write a program that goes through all possibilities and try them one by one, maybe finding more than one solution if more exist. This requires knowing how to approach the problem and define an explicit and efficient search strategy, which can be difficult to find and implement.
4. When we don't really know what to do, we can start modelling the puzzle / state, and use stochastic search to find a solution. This requires a set of good building blocks and tools that the stochastic search can rely on.

## Strategy

In this notebook, I would like to try the last approach, using a stochastic approach. 
To prove that the solution is correct, the best way is probably to draw it right away, and visually verify the colors match. This is what we would do with the real puzzle.

Since we will already be drawing the puzzle, we can push one step further to make an interactive animation of the stochastic search.

This is where we introduce [p5.js](https://p5js.org), a web-based port of the rendering framework Processing, for easy to make graphics right here in the browser.

Steps:

1. Model the puzzle by encoding the colors to numbers and arranging them into lists.
2. Define a function, which draws a state on a canvas. A state is defined as the big triangle and a permutation of the 16 small triangles potentially rotated.
3. Define a function, which given a list of actions (permutations of triangles, rotation of triangles), animate them on a canvas.
4. Define a small helper for genetic algorithms.
5. Define a function that uses the helper to search for the solution, logging each step taken (permutation, rotation) to a list of actions.
6. Use the previously defined functions to animated the search and display the final solution

## Tools

For this kind of exploratory problem, we will rely on a few Python packages. However this time, I want to introduce them one by one when needed, to follow the explorative approach.

## Model

This is perhaps the most tedious step, as it requires listing the different pieces with the proper order for the colors.

### Triangles

In total, we have 16 triangles, and 6 colors: **white, blue, yellow, green, black, red**.

In [1]:
N_TRIANGLES = 16
IDS = list(range(N_TRIANGLES))
N_COLORS = 6
WHITE, BLUE, YELLOW, GREEN, BLACK, RED = range(N_COLORS)

# list the pieces, turning anti-clockwise for the colors
# The number at the last position of the tuple indicate the number
# of identical pieces
triangles_count = [
    (WHITE, BLUE, BLUE, 1),
    (WHITE, YELLOW, GREEN, 2),
    (WHITE, BLACK, BLUE, 2),
    (WHITE, GREEN, RED, 1),
    (WHITE, RED, YELLOW, 1),
    (WHITE, WHITE, BLUE, 1),
    (BLACK, GREEN, RED, 1),
    (BLACK, RED, GREEN, 2),
    (BLACK, BLACK, GREEN, 1),
    (BLACK, GREEN, YELLOW, 1),
    (BLACK, YELLOW, BLUE, 1),
    (GREEN, RED, YELLOW, 1),
    (BLUE, GREEN, YELLOW, 1)
]

assert N_TRIANGLES == sum(t[-1] for t in triangles_count)

The triangles were listed correctly, let's unpack them into a flat list. Additionally, we add a number (0, 1 or 2) corresponding to the current rotation (initially 0)

In [2]:
triangles = tuple([t[:-1] for t in triangles_count for times in range(t[-1])])
print(triangles)

assert N_TRIANGLES == len(triangles)

((0, 1, 1), (0, 2, 3), (0, 2, 3), (0, 4, 1), (0, 4, 1), (0, 3, 5), (0, 5, 2), (0, 0, 1), (4, 3, 5), (4, 5, 3), (4, 5, 3), (4, 4, 3), (4, 3, 2), (4, 2, 1), (3, 5, 2), (1, 3, 2))


This is our immutable list of triangles (tuples), that we can index by an integer from 0 to 15.

### Board

What needs to be done now is to define the board with the fixed colors. Together with the triangles, they define the **state** of the puzzle.

In [3]:
import json

class Board:
    
    TRIANGLES = triangles
    LEFT = (WHITE, RED, WHITE, YELLOW)
    RIGHT = (BLUE, RED, GREEN, BLACK)
    BOTTOM = (GREEN, GREEN, WHITE, GREEN)
    
    def __str__(self):
        return json.dumps({
            'TRIANGLES': Board.TRIANGLES,
            'LEFT': Board.LEFT,
            'RIGHT': Board.RIGHT,
            'BOTTOM': Board.BOTTOM,
        })


class Permutation(Board):
    def __init__(self, permutation, states=[]):
        self.permutation = list(permutation)
        self.repr = [triangles[p[0]] + (p[1],) for p in self.permutation]
        self.states = states
        
    def __str__(self):
        base = json.loads(super().__str__())
        base.update({
            'repr': self.repr,
            'permutation': self.permutation,
            'states': self.states,
        })
        return json.dumps(base)

Create global storage to write states and variables from Python, and access them from Javascript

In [4]:
%%javascript
window.kv = {};

<IPython.core.display.Javascript object>

In [5]:
from IPython.display import Javascript

def send_to_js(key, value):
    display(Javascript(f'window.kv.{key} = {value};'))

In [6]:
send_to_js('defaultBoard', Permutation([i, 0] for i in range(N_TRIANGLES)))

<IPython.core.display.Javascript object>

## Drawing the state

Since the Jupyter Notebook is basically a webpage, we can directly use battle-tested web libraries to draw shapes on a canvas. One of my favorite is [p5.js](https://p5js.org), let's see how we can use it here.

In [7]:
%%javascript
require.config({
    paths: {
        'p5': 'https://cdnjs.cloudflare.com/ajax/libs/p5.js/0.5.16/p5.min',
        'p5.dom': 'https://cdnjs.cloudflare.com/ajax/libs/p5.js/0.5.16/addons/p5.dom.min'
    }
});

<IPython.core.display.Javascript object>

Jupyter uses [require.js](http://requirejs.org/) under the hood, and the good thing is that we make use of it for our own purposes.

We can for example define small modules relying on p5.js, that will in the end be assembled together to form the bigger picture.

In [8]:
%%javascript

window.defineModule = function (name, dependencies, module) {
    // force the recreation of the module
    // (when re-executing a cell for example)
    require.undef(name);
    
    define(name, dependencies, module);
};

window.createSketch = function (cell, name, dependencies, module) {
    // global store for sketches, lazily created
    window.sketches = window.sketches || {};
    
    // remove existing if it already exists
    $(`${name}`).remove();
    cell.append(`<div id="${name}" style="text-align: center;"></div>`);
    
    // stop the existing sketch
    let sketch = window.sketches[name];
    if (sketch) {
        clearTimeout(sketch.timeout);
        sketch.remove();
    }
    
    require(['p5', 'p5.dom'], (p5, p5dom) => {
        require(dependencies, function(...deps) {
            let sketch = module(...deps);
            window.sketches[name] = new p5(sketch, name);
            console.log(`sketch ${name} created`);
        })
    });
}

<IPython.core.display.Javascript object>

The `defineModule` javascript function can be used right away to create a module containing the list of settings.

In [9]:
%%javascript

defineModule('settings', [], () => {
    let N_TRIANGLES = 16;
    let COLORS = ['#fff', '#00f', '#ff0', '#0f0', '#000', '#f00'];
    let R = 50;
    let r = R / 2;
    let CR = r / 2;
    let OFFSET_X = R * Math.sqrt(3) * 0.5;
    let OFFSET_Y = 1.5 * R;
    
    return {N_TRIANGLES, COLORS, R, r, CR, OFFSET_X, OFFSET_Y};
})

<IPython.core.display.Javascript object>

With these parameters defined, we can create a `triangles` module that will be used to draw one triangle on the screen.

In [10]:
%%javascript

defineModule('triangles', ['settings'], (settings) => {
    let {COLORS, R, r, CR, OFFSET_X, OFFSET_Y} = settings;
    
    function _getPoints(n, startAngle, radius) {
        let points = [];
        let da = 2 * Math.PI / n;
        for (let i = 0; i < n; i++) {
            let angle = startAngle - i * da;
            let x = radius * Math.cos(angle);
            let y = radius * Math.sin(angle);
            points.push(x);
            points.push(y);
        }
        return points;
    }
    
    return (p) => {
        return {
            genTrianglesPos: function* () {
                let id = 0;
                for (let row = 0; row < 4; row++) {
                    let n_row = 2 * row + 1;
                    for (let col = 0; col < n_row; col++) {
                        let x = (col - row) * OFFSET_X;
                        let flip = (id + row) % 2;
                        let y = row * OFFSET_Y + ((flip == 1) ? -R/2 : 0);
                        yield {id, x, y, flip, row, col, n_row};
                        id++;
                    }
                }
            },
            drawTriangle: function (colors, x, y, rotation, flip=0) {
                p.fill('#825201'); // wood color

                let n = colors.length;
                p.push();
                p.translate(x, y);
                p.rotate(-rotation * p.TWO_PI / 3 + flip * p.PI);
                
                p.triangle(..._getPoints(n, Math.PI / 6, R));

                let circles = _getPoints(n, Math.PI / 2, 1.25 * CR);
                for (let i = 0; i < n; i++) {
                    let xx = circles[2*i];
                    let yy = circles[2*i+1];
                    let color = COLORS[colors[i]];
                    p.fill(color);
                    p.ellipse(xx, yy, CR);
                }
                p.pop();
            }
        };
    };
});

<IPython.core.display.Javascript object>

### Static Board

Let's start small and focus on drawing a static board. This is were the different modules defined above become useful, as we can now depend on them.

In [11]:
%%javascript

defineModule('staticBoard', ['settings', 'triangles'], (Settings, Triangles) => {
    let {COLORS, R, CR, OFFSET_X, OFFSET_Y} = Settings;
    
    return (p) => {
        let W = p.width;
        let H = p.height;
        let triangles = Triangles(p);
        
        function _drawStaticColors (board) {
            for (let {id, x, y, row, col, n_row} of triangles.genTrianglesPos()) {
                if (col === 0) {
                    let left = COLORS[board.LEFT[row]];
                    let right = COLORS[board.RIGHT[row]];
                    p.fill(left);
                    p.ellipse(x - OFFSET_X, y - R / 2, CR);
                    p.fill(right);
                    p.ellipse(x + n_row * OFFSET_X, y - R / 2, CR);
                }
                          
                if (row === 3 && col % 2 == 0) {
                    p.fill(COLORS[board.BOTTOM[parseInt(col / 2, 10)]]);
                    p.ellipse(x, 3.75*OFFSET_Y, CR);
                }
            }
        }
        
        return {
            drawStaticColors: _drawStaticColors,
            drawBoard: (boardName) => {
                let board = window.kv[boardName];
                let repr = board.repr;
                
                for (let {id, x, y, flip} of triangles.genTrianglesPos()) {
                    let [a, b, c, rot] = repr[id];
                    p.push();
                    triangles.drawTriangle([a, b, c], x, y, rot, flip);
                    p.pop();
                }
                
                _drawStaticColors(board);
            }
        };
    };
});

<IPython.core.display.Javascript object>

Creating the sketch is now just a matter of putting the previous pieces together:

In [12]:
%%javascript

window.drawStaticBoard = function (name, boardName, element) {
    createSketch(element, name, ['staticBoard'], function (StaticBoard) {
        let W = 400;
        let H = 400;

        return function(p) {
            let staticBoard = StaticBoard(p);

            p.setup = function(){
                p.createCanvas(W, H);
                p.noLoop();
            }

            p.draw = function () {
                p.background('#ddd');
                p.push();
                p.translate(W / 2, H / 4);
                staticBoard.drawBoard(boardName);
                p.pop();
            }
        };
    });
}

<IPython.core.display.Javascript object>

In [13]:
send_to_js('staticBoardTest', Permutation([i, 0] for i in range(N_TRIANGLES)))

<IPython.core.display.Javascript object>

In [14]:
%%javascript

drawStaticBoard('drawTriangles', 'staticBoardTest', element)

<IPython.core.display.Javascript object>

### Rotating the triangles

You must have noticed the magic `0` number as the last element of the triangle tuple.
This number can take the values `[0, 1, 2]`. To make it simple, we order the three colors of the triangles from 0 to 2, with 0 corresponding to the color at the "bottom" of the triangle.

Difficult to visualize? No problem, we can create a new sketch for that purpose!

In [15]:
%%javascript

require.config({
    paths: {
        tween: 'https://cdnjs.cloudflare.com/ajax/libs/tween.js/17.1.1/Tween.min'
    }
});

<IPython.core.display.Javascript object>

In [16]:
%%javascript

createSketch(element, 'rotate-triangle-demo', ['tween', 'triangles'], function (Tween, Triangles) {
    let W = 300; let H = 150;
    
    return (p) => {
        let obj = { angle: 0 };
        let T = Triangles(p);
        let rotateButton;
        
        let tweenGroup = new Tween.Group();
        let t = new Tween.Tween(obj, tweenGroup)
                    .to({angle: "+" + (p.TWO_PI / 3)}, 500)
                    .easing(Tween.Easing.Quadratic.InOut)
                    .onStart(() => t.running = true)
                    .onComplete(() => t.running = false)
        
        function rotate () {
            if (t.running) return;
            t.start();
        }

        p.setup = function(){
            p.createCanvas(W, H);
            rotateButton = p.createButton('Rotate');
            rotateButton.style('color', '#000');
            rotateButton.mousePressed(rotate);
            rotateButton.position(150, 10);
        }

        p.draw = function () {
            tweenGroup.update();
            p.background('#ddd');
            p.translate(W / 3, H / 2);
            p.push();
            p.rotate(obj.angle);
            T.drawTriangle([0, 1, 2], 0);
            p.pop();
            p.push();
            p.translate(W / 3, 0);
            p.rotate(-obj.angle);
            T.drawTriangle([3, 4, 5], 0);
            p.pop();
        }
    };
});

<IPython.core.display.Javascript object>

### Animated Board

It would be more interesting to actually visualize the entire solving process in an animated fashion.

Similar to the previous examples, we can define a new require module called `animatedBoard`.

In [17]:
%%javascript

defineModule('animatedBoard',
             ['settings', 'staticBoard', 'triangles', 'tween'],
             (Settings, StaticBoard, Triangles, Tween) => {
    
    let {N_TRIANGLES} = Settings;
    
    return (p) => {
        let tweenGroup = new Tween.Group();
        let globalTime = 0;
        
        let staticBoard = StaticBoard(p);
        let triangles = Triangles(p);
        
        let [it, animationSpeed, baseSpeed, step] = [0, 1000, 1000, 1];
        let board = null;
        
        // pre-compute the positions of the triangles
        let triangleGen = triangles.genTrianglesPos();
        let pos = _.range(N_TRIANGLES).map(i => _.pick(triangleGen.next().value, ['x', 'y', 'flip']));
        let start = _.range(N_TRIANGLES).map(i => ({x: 0, y: 0, r: 0, f: 0}));
        let end = _.range(N_TRIANGLES).map(i => ({x: 0, y: 0, r: 0, f: 0}));
        let moving = [];

        function transitionState(curr) {
            let states = board.states;
            let [from, to] = [states[curr], states[curr + 1]];
            _.range(N_TRIANGLES).forEach(i => {
                _.assign(start[i], {x: pos[i].x, y: pos[i].y, r: from[i][1], f: pos[i].flip});
                let pp = i;
                if (from[i][0] !== to[i][0]) {
                    pp = states[curr + 1].findIndex((e) => e[0] === from[i][0]);
                }
                _.assign(end[i], {x: pos[pp].x, y: pos[pp].y, r: to[pp][1], f: pos[pp].flip});
            });

            moving = [];
            start.forEach((a, i) => {
                let b = end[i];
                if (a.x != b.x || a.y != b.y || a.r != b.r) {
                    moving.push(i);
                    new Tween.Tween(a, tweenGroup)
                        .to({x: b.x, y: b.y, r: b.r, f: b.f}, animationSpeed)
                        .easing(Tween.Easing.Quadratic.InOut)
                        .start(globalTime)
                }
            });
            
            new Tween.Tween({t: 0}, tweenGroup)
                .to({t: animationSpeed}, animationSpeed)
                .onComplete(() => {
                    if (it >= states.length - 2) return;
                    updateIterationNumber(it+1);
                    transitionState(curr + Math.min(step, states.length - it - 1));
                })
                .start(globalTime);
            
        }
        
        // controls
        let playButton;
        let running = true;
        let iterationNumber;
        
        function updateIterationNumber(newIt) {
            it = newIt;
            iterationNumber.value(it);
        }
        
        function setState(stateId) {
            tweenGroup.removeAll();
            it = Math.max(0, Math.min(stateId, board.states.length - 2));
            updateIterationNumber(it);
            transitionState(it);
            running = false;
        }

        return {
            init: (boardName) => {
                board = window.kv[boardName];
                playButton = p.createButton('❚❚');
                playButton.style('color', '#000');
                playButton.style('width', '50px');
                playButton.mousePressed(() => running = !running);
                playButton.position(200, 10);
                
                let prevButton = p.createButton('◄');
                prevButton.style('color', '#000');
                prevButton.style('width', '50px');
                prevButton.mousePressed(() => setState(it-1));
                prevButton.position(150, 10);
                
                let nextButton = p.createButton('►');
                nextButton.style('color', '#000');
                nextButton.style('width', '50px');
                nextButton.mousePressed(() => setState(it+1));
                nextButton.position(250, 10);
                
                _.range(3).forEach(i => {
                    let speedRatio = Math.pow(10, i);
                    let speedButton = p.createButton(`x${speedRatio}`);
                    speedButton.position(150 + 50 * i, 40);
                    speedButton.style('width', '50px');
                    speedButton.style('color', '#000');
                    speedButton.mousePressed(() => animationSpeed = baseSpeed / speedRatio);
                });
                
                let iterationText = p.createElement('div', 'Iteration:');
                iterationText.position(150, 80);
                iterationText.style('width', '60px');
                iterationText.style('color', '#fff');
                
                iterationNumber = p.createInput();
                iterationNumber.position(210, 80);
                iterationNumber.style('width', '50px');
                iterationNumber.style('color', '#000');
                
                let iterationSubmit = p.createButton('Set');
                iterationSubmit.position(260, 80);
                iterationSubmit.style('width', '40px');
                iterationSubmit.style('color', '#000');
                iterationSubmit.mousePressed(() => setState(parseInt(iterationNumber.value(), 10) || 0));
                
            },
            startAnimation: (startIteration, speed) => {
                it = startIteration;
                baseSpeed = speed;
                animationSpeed = speed;
                transitionState(0);
            },
            draw: () => {
                playButton.html(running ? '❚❚' : '►');
                if (running) {
                    tweenGroup.update(globalTime);  
                    globalTime += Math.min(1000 / p.frameRate(), 33);
                }
                
                p.fill('#000');
                p.textSize(24);
                p.text(`Iteration: ${it}`, 10, 30);
                
                p.translate(p.width / 2, p.height / 4);
                let allTriangles = _.range(N_TRIANGLES);
                let staticTriangles = _.difference(allTriangles, moving);
                [staticTriangles, moving].forEach(bucket => {
                    bucket.forEach(id => {
                        let [triangleId, rot] = board.states[it][id];
                        let [a, b, c] = board.TRIANGLES[triangleId];
                        let {x, y, r, f} = start[id];
                        p.push();
                        triangles.drawTriangle([a, b, c], x, y, r, f);
                        p.pop();
                    });
                });
                staticBoard.drawStaticColors(board);
            }
        }
    };
});

<IPython.core.display.Javascript object>

In [18]:
%%javascript

window.animateBoard = function (name, boardName, element, startIteration=0, speed=1000) {
    createSketch(element, name, ['animatedBoard'], (AnimatedBoard) => {
        let W = 400; let H = 400;

        return function(p) {
            let board = AnimatedBoard(p);
            
            p.setup = function () {
                p.createCanvas(W, H);
                board.init(boardName);
                board.startAnimation(startIteration, speed);
            }

            p.draw = function () {
                p.background('#ddd');
                board.draw();
            }
        };
    });
}

<IPython.core.display.Javascript object>

Let's create a test Permutation for testing.

In [19]:
board = Permutation(
    [[i, 0] for i in range(N_TRIANGLES)],
    (
        ((7,1),(3,0),(11,1),(8,1),(0,1),(15,0),(1,1),(12,1),(10,0),(14,0),(13,1),(5,1),(4,1),(2,0),(9,0),(6,1)),
        ((7,1),(3,0),(11,1),(8,1),(0,1),(15,0),(1,1),(12,1),(10,0),(14,0),(13,1),(5,1),(4,1),(6,0),(9,0),(2,0)),
        ((7,1),(3,0),(11,1),(8,1),(0,1),(15,0),(14,0),(12,1),(10,0),(1,1),(13,1),(5,1),(4,1),(6,0),(9,0),(2,0)),
        ((3,2),(7,1),(11,1),(8,1),(0,1),(15,0),(14,0),(12,1),(10,0),(1,1),(13,1),(5,1),(4,1),(6,0),(9,0),(2,0)),
        ((3,2),(4,0),(11,1),(8,1),(0,1),(15,0),(14,0),(12,1),(10,0),(1,1),(13,1),(5,1),(7,0),(6,0),(9,0),(2,0))
    ) * 100
)
send_to_js('testAnimatedBoard', board)

<IPython.core.display.Javascript object>

In [20]:
%%javascript

animateBoard('animatedBoard', 'testAnimatedBoard', element, 0, 5000)

<IPython.core.display.Javascript object>

## Stochastic Search

### Scoring function

The key part when using a stochastic search approach (Genetic Algorithms, Simulated Annealing) is the **scoring function** (also called fitness function). The function should return a score for a given state, to rate the quality of the current solution.

For this puzzle, there can be more than one valid scoring function. A simple one can be defined as the number of triangles correctly placed on the board. To know if a triangle is correctly placed, all its colors must match the ones from the adjacent neighbors.

#### Generating the function

Similar to one of the previous puzzles (see [Aristote Number Puzzle](https://jtp.io/2017/01/12/aristotle-number-puzzle.html)), we can generate the scoring function instead of going through the logic every time.

The scoring function should give a good indication of how good the placement of the triangles is.

In [21]:
import numpy as np

cumsum = np.cumsum([2 * k + 1 for k in range(4)])
cumsum

array([ 1,  4,  9, 16])

In [22]:
def pack(j, i, c):
    return sum(2 * k + 1 for k in range(j)) + i

def tri(j, i, c):
    n = pack(j, i, c)
    return f'triangles[p[{n}][0]][{c}-p[{n}][1]]'

def unpack(triangle_id):
    j = next(i for i, s in enumerate(cumsum) if s > triangle_id)
    i = 2 * j + 1
    k = max(0, triangle_id - cumsum[max(0, j - 1)] )
    return (j, i, k)

In [23]:
def generate_score_triangle(triangle_id, reverse=False):
    j, i, k = unpack(triangle_id)
    code = []
    if k == 0:
        # on the left edge
        code.append(f'{tri(j, k, 2)} == Board.LEFT[{j}]')

    if k == i - 1:
        # on the right edge
        code.append(f'{tri(j, k, 1)} == Board.RIGHT[{j}]')

    if j == 3 and k % 2 == 0:
        # on the bottom edge
        code.append(f'{tri(j, k, 0)} == Board.BOTTOM[{k//2}]')

    if k % 2 == 0:
        # normal orientation (facing up)
        if j < 3:
            # match with line below
            code.append(f'{tri(j, k, 0)} == {tri(j + 1, k + 1, 0)}')
        if k > 0:
            # match with triangle on the left
            code.append(f'{tri(j, k, 2)} == {tri(j, k - 1, 2)}')
        if k < i - 1:
            # match with triangle on the right
            code.append(f'{tri(j, k, 1)} == {tri(j, k + 1, 1)}')
            
    if k % 2 == 1 and reverse:
        # reverse orientation (facing down)
        # match with line above
        code.append(f'{tri(j, k, 0)} == {tri(j - 1, k - 1, 0)}')
        # match with triangle on the left
        code.append(f'{tri(j, k, 2)} == {tri(j, k - 1, 2)}')
        # match with triangle on the right
        code.append(f'{tri(j, k, 1)} == {tri(j, k + 1, 1)}')
        
    return code

generate_score_triangle(2, True)

['triangles[p[2][0]][0-p[2][1]] == triangles[p[0][0]][0-p[0][1]]',
 'triangles[p[2][0]][2-p[2][1]] == triangles[p[1][0]][2-p[1][1]]',
 'triangles[p[2][0]][1-p[2][1]] == triangles[p[3][0]][1-p[3][1]]']

In [24]:
def generate_all_triangle_scoring():
    code = []
    for n in range(N_TRIANGLES):
        code += [f'lambda p: sum(({", ".join(generate_score_triangle(n, True))}))']
        
    return 'triangle_scoring = [\n\t\t' + ',\n\t\t'.join(code) + ']'

code = generate_all_triangle_scoring()
print('Code:')
print(code[:1000])
print('Evaluating the code...')
exec(code)
assert 'triangle_scoring' in globals()
print('done.')

Code:
triangle_scoring = [
		lambda p: sum((triangles[p[0][0]][2-p[0][1]] == Board.LEFT[0], triangles[p[0][0]][1-p[0][1]] == Board.RIGHT[0], triangles[p[0][0]][0-p[0][1]] == triangles[p[2][0]][0-p[2][1]])),
		lambda p: sum((triangles[p[1][0]][2-p[1][1]] == Board.LEFT[1], triangles[p[1][0]][0-p[1][1]] == triangles[p[5][0]][0-p[5][1]], triangles[p[1][0]][1-p[1][1]] == triangles[p[2][0]][1-p[2][1]])),
		lambda p: sum((triangles[p[2][0]][0-p[2][1]] == triangles[p[0][0]][0-p[0][1]], triangles[p[2][0]][2-p[2][1]] == triangles[p[1][0]][2-p[1][1]], triangles[p[2][0]][1-p[2][1]] == triangles[p[3][0]][1-p[3][1]])),
		lambda p: sum((triangles[p[3][0]][1-p[3][1]] == Board.RIGHT[1], triangles[p[3][0]][0-p[3][1]] == triangles[p[7][0]][0-p[7][1]], triangles[p[3][0]][2-p[3][1]] == triangles[p[2][0]][2-p[2][1]])),
		lambda p: sum((triangles[p[4][0]][2-p[4][1]] == Board.LEFT[2], triangles[p[4][0]][0-p[4][1]] == triangles[p[10][0]][0-p[10][1]], triangles[p[4][0]][1-p[4][1]] == triangles[p[5][0]][1-p[5][1

In [25]:
def generate_scoring_function():
    code = []
    for n in range(N_TRIANGLES):
        _, _, k = unpack(n)
        if k % 2 != 0:
            continue
        code += [',\n\t\t'.join(generate_score_triangle(n))]
    return 'def scoring (p):\n\treturn sum((\n\t\t' + ',\n\t\t'.join(code) + '\n\t))'


code = generate_scoring_function()
print('Code:')
print(code[:1000])
print('Evaluating the code...')
exec(code)
assert 'scoring' in globals()
print('done.')

Code:
def scoring (p):
	return sum((
		triangles[p[0][0]][2-p[0][1]] == Board.LEFT[0],
		triangles[p[0][0]][1-p[0][1]] == Board.RIGHT[0],
		triangles[p[0][0]][0-p[0][1]] == triangles[p[2][0]][0-p[2][1]],
		triangles[p[1][0]][2-p[1][1]] == Board.LEFT[1],
		triangles[p[1][0]][0-p[1][1]] == triangles[p[5][0]][0-p[5][1]],
		triangles[p[1][0]][1-p[1][1]] == triangles[p[2][0]][1-p[2][1]],
		triangles[p[3][0]][1-p[3][1]] == Board.RIGHT[1],
		triangles[p[3][0]][0-p[3][1]] == triangles[p[7][0]][0-p[7][1]],
		triangles[p[3][0]][2-p[3][1]] == triangles[p[2][0]][2-p[2][1]],
		triangles[p[4][0]][2-p[4][1]] == Board.LEFT[2],
		triangles[p[4][0]][0-p[4][1]] == triangles[p[10][0]][0-p[10][1]],
		triangles[p[4][0]][1-p[4][1]] == triangles[p[5][0]][1-p[5][1]],
		triangles[p[6][0]][0-p[6][1]] == triangles[p[12][0]][0-p[12][1]],
		triangles[p[6][0]][2-p[6][1]] == triangles[p[5][0]][2-p[5][1]],
		triangles[p[6][0]][1-p[6][1]] == triangles[p[7][0]][1-p[7][1]],
		triangles[p[8][0]][1-p[8][1]] == Board.RIGHT[

In [26]:
def generate_scoring_function():
    
    def tri(j, i, c):
        n = sum(2 * k + 1 for k in range(j)) + i
        return f'triangles[p[{n}][0]][{c}-p[{n}][1]]'
    
    code = []
    for j in range(4):
        i = 2 * j + 1
        for k in range(i):
            if k == 0:
                # on the left edge
                code.append(f'{tri(j, k, 2)} == Board.LEFT[{j}]')
                
            if k == i - 1:
                # on the right edge
                code.append(f'{tri(j, k, 1)} == Board.RIGHT[{j}]')
        
            if j == 3 and k % 2 == 0:
                # on the bottom edge
                code.append(f'{tri(j, k, 0)} == Board.BOTTOM[{k//2}]')
                
            if k % 2 == 0:
                # normal orientation (facing up)
                if j < 3:
                    # match with line below
                    code.append(f'{tri(j, k, 0)} == {tri(j + 1, k + 1, 0)}')
                if k > 0:
                    # match with triangle on the left
                    code.append(f'{tri(j, k, 2)} == {tri(j, k - 1, 2)}')
                if k < i - 1:
                    # match with triangle on the right
                    code.append(f'{tri(j, k, 1)} == {tri(j, k + 1, 1)}')
        
    return 'def scoring (p):\n\treturn sum((\n\t\t' + ',\n\t\t'.join(code) + '\n\t))'

code = generate_scoring_function()
print('Code:')
print(code[:1000])
print('Evaluating the code...')
exec(code)
assert 'scoring' in globals()
print('done.')

Code:
def scoring (p):
	return sum((
		triangles[p[0][0]][2-p[0][1]] == Board.LEFT[0],
		triangles[p[0][0]][1-p[0][1]] == Board.RIGHT[0],
		triangles[p[0][0]][0-p[0][1]] == triangles[p[2][0]][0-p[2][1]],
		triangles[p[1][0]][2-p[1][1]] == Board.LEFT[1],
		triangles[p[1][0]][0-p[1][1]] == triangles[p[5][0]][0-p[5][1]],
		triangles[p[1][0]][1-p[1][1]] == triangles[p[2][0]][1-p[2][1]],
		triangles[p[3][0]][1-p[3][1]] == Board.RIGHT[1],
		triangles[p[3][0]][0-p[3][1]] == triangles[p[7][0]][0-p[7][1]],
		triangles[p[3][0]][2-p[3][1]] == triangles[p[2][0]][2-p[2][1]],
		triangles[p[4][0]][2-p[4][1]] == Board.LEFT[2],
		triangles[p[4][0]][0-p[4][1]] == triangles[p[10][0]][0-p[10][1]],
		triangles[p[4][0]][1-p[4][1]] == triangles[p[5][0]][1-p[5][1]],
		triangles[p[6][0]][0-p[6][1]] == triangles[p[12][0]][0-p[12][1]],
		triangles[p[6][0]][2-p[6][1]] == triangles[p[5][0]][2-p[5][1]],
		triangles[p[6][0]][1-p[6][1]] == triangles[p[7][0]][1-p[7][1]],
		triangles[p[8][0]][1-p[8][1]] == Board.RIGHT[

In [27]:
MAX_SCORING = 30
print(f'{MAX_SCORING} equalities to solve')

30 equalities to solve


We can now evaluate the scoring function on a real state, and manually count the number of matches to see if they are equali

In [28]:
state = [[i, 0] for i in range(N_TRIANGLES)]
np.random.shuffle(state)
scoring(state)

7

In [29]:
send_to_js('randomBoard', Permutation(state))
state

<IPython.core.display.Javascript object>

[[11, 0],
 [1, 0],
 [4, 0],
 [10, 0],
 [8, 0],
 [13, 0],
 [2, 0],
 [15, 0],
 [9, 0],
 [3, 0],
 [12, 0],
 [14, 0],
 [6, 0],
 [0, 0],
 [7, 0],
 [5, 0]]

In [30]:
%%javascript

drawStaticBoard('testPermutations', 'randomBoard', element)

<IPython.core.display.Javascript object>

## Simulated Annealing

I you happen to read some of the [previous](https://jtp.io/2017/05/10/nine-color-cube.html) [articles](https://jtp.io/2017/01/12/aristotle-number-puzzle.html), you will notice that I am quite a big fan of a technique called **Simulated Annealing**, mostly because of its simple implementation.

In [31]:
import math
from copy import deepcopy

class SimulatedAnnealingSolver:
    
    def __init__(self, seed=0):
        self.seed = seed
        self.rng = np.random.RandomState()
        self.reset_state()
        
    def reset_state(self):
        self.it = 0
        self.rng.seed(self.seed)
        self.log = []
        
    def _cost(self, permutation):
        return MAX_SCORING - scoring(permutation)

    def _neighbor(self, permutation):
        neighbor = deepcopy(permutation)
        i, j = self.rng.choice(IDS, 2)
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
        
        for k in [i, j]:
            original_rot = neighbor[k][1]
            best_rot = 0
            best_score = 0
            for rot in range(3):
                neighbor[k][1] = rot
                score = scoring(neighbor)
                if score > best_score:
                    best_score = score
                    best_rot = rot

            neighbor[k][1] = best_rot

        return neighbor

    def run(self, start_solution=None, max_it=50000, verbose=False):
        self.reset_state()
        solution = start_solution or self.rng.permutation([[i, self.rng.randint(0, 2)] for i in IDS]).tolist()
        curr_cost = self._cost(solution)
        temp, alpha = 1.0, 0.992
        while curr_cost > 0 and self.it < max_it:
            new_solution = self._neighbor(solution)
            new_cost = self._cost(new_solution)
            diff = new_cost - curr_cost
            if diff <= 0 or math.exp(-2*diff / temp) > self.rng.rand():
                solution = new_solution
                curr_cost = new_cost
                self.log.append(deepcopy(new_solution))

            if self.it % 500 == 0:
                if temp > 0.4:
                    temp *= alpha
            
            if verbose and self.it % 1000 == 0:
                print(curr_cost)
                
            self.it += 1

        return solution

In [32]:
solver = SimulatedAnnealingSolver(1)
res = solver.run(max_it=100000, verbose=False)
score = scoring(res)
if score == MAX_SCORING:
    print('Solution found after', solver.it, 'iterations')
    print(res)
    print('Actions:')
    send_to_js('simulatedAnnealing', Permutation(res, solver.log))
else:
    print('No solution found, score:', score)

Solution found after 6593 iterations
[[3, 2], [14, 1], [11, 2], [8, 2], [0, 2], [15, 1], [9, 0], [5, 2], [2, 2], [1, 1], [4, 1], [12, 2], [13, 0], [7, 0], [6, 1], [10, 1]]
Actions:


<IPython.core.display.Javascript object>

In [33]:
%%javascript

drawStaticBoard('solution-sa', 'simulatedAnnealing', element)

<IPython.core.display.Javascript object>

In [34]:
%%javascript

animateBoard('simulated-annealing', 'simulatedAnnealing', element, 0, 1000)

<IPython.core.display.Javascript object>

## Recursive

In [35]:
N_TRIANGLES

16

In [36]:
def generate_valid_triangle(triangle_id, reverse=False):
    j, i, k = unpack(triangle_id)
    code = []
    if k == 0:
        # on the left edge
        code.append(f'{tri(j, k, 2)} == Board.LEFT[{j}]')

    if k == i - 1:
        # on the right edge
        code.append(f'{tri(j, k, 1)} == Board.RIGHT[{j}]')

    if j == 3 and k % 2 == 0:
        # on the bottom edge
        code.append(f'{tri(j, k, 0)} == Board.BOTTOM[{k//2}]')

    if k % 2 == 0:
        # normal orientation (facing up)
        if j < 3:
            # match with line below
            code.append(f'(not p[{pack(j + 1, k + 1, 0)}] or {tri(j, k, 0)} == {tri(j + 1, k + 1, 0)})')
        if k > 0:
            # match with triangle on the left
            code.append(f'(not p[{pack(j, k - 1, 2)}] or {tri(j, k, 2)} == {tri(j, k - 1, 2)})')
        if k < i - 1:
            # match with triangle on the right
            code.append(f'(not p[{pack(j, k + 1, 1)}] or {tri(j, k, 1)} == {tri(j, k + 1, 1)})')
            
    if k % 2 == 1 and reverse:
        # reverse orientation (facing down)
        # match with line above
        code.append(f'(not p[{pack(j - 1, k - 1, 0)}] or {tri(j, k, 0)} == {tri(j - 1, k - 1, 0)})')
        # match with triangle on the left
        code.append(f'(not p[{pack(j, k - 1, 2)}] or {tri(j, k, 2)} == {tri(j, k - 1, 2)})')
        # match with triangle on the right
        code.append(f'(not p[{pack(j, k + 1, 1)}] or {tri(j, k, 1)} == {tri(j, k + 1, 1)})')
        
    return code

generate_valid_triangle(0, True)

['triangles[p[0][0]][2-p[0][1]] == Board.LEFT[0]',
 'triangles[p[0][0]][1-p[0][1]] == Board.RIGHT[0]',
 '(not p[2] or triangles[p[0][0]][0-p[0][1]] == triangles[p[2][0]][0-p[2][1]])']

In [37]:
def generate_valid_function():
    code = []
    for n in range(N_TRIANGLES):
        _, _, k = unpack(n)
        if k % 2 != 0:
            continue
        code += [f"(not p[{n}] or ({' and '.join(generate_valid_triangle(n))}))"]
    return 'def is_valid (p):\n\treturn ' + ' and '.join(code) + '\n\t'


code = generate_valid_function()
print('Code:')
print(code)
print('Evaluating the code...')
exec(code)
assert 'scoring' in globals()
print('done.')

Code:
def is_valid (p):
	return (not p[0] or (triangles[p[0][0]][2-p[0][1]] == Board.LEFT[0] and triangles[p[0][0]][1-p[0][1]] == Board.RIGHT[0] and (not p[2] or triangles[p[0][0]][0-p[0][1]] == triangles[p[2][0]][0-p[2][1]]))) and (not p[1] or (triangles[p[1][0]][2-p[1][1]] == Board.LEFT[1] and (not p[5] or triangles[p[1][0]][0-p[1][1]] == triangles[p[5][0]][0-p[5][1]]) and (not p[2] or triangles[p[1][0]][1-p[1][1]] == triangles[p[2][0]][1-p[2][1]]))) and (not p[3] or (triangles[p[3][0]][1-p[3][1]] == Board.RIGHT[1] and (not p[7] or triangles[p[3][0]][0-p[3][1]] == triangles[p[7][0]][0-p[7][1]]) and (not p[2] or triangles[p[3][0]][2-p[3][1]] == triangles[p[2][0]][2-p[2][1]]))) and (not p[4] or (triangles[p[4][0]][2-p[4][1]] == Board.LEFT[2] and (not p[10] or triangles[p[4][0]][0-p[4][1]] == triangles[p[10][0]][0-p[10][1]]) and (not p[5] or triangles[p[4][0]][1-p[4][1]] == triangles[p[5][0]][1-p[5][1]]))) and (not p[6] or ((not p[12] or triangles[p[6][0]][0-p[6][1]] == triangles[p[12][

In [38]:
state = [None for i in range(N_TRIANGLES)]
state[0] = [3, 0]
print(state)
is_valid(state)

[[3, 0], None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


False

In [39]:
def solve_recursive():
    board = [None] * N_TRIANGLES
    used = [0] * N_TRIANGLES
    
    def place(i):
        if i == N_TRIANGLES:
            # base case, all the pieces are on the board
            return True
        
        for j in range(0, N_TRIANGLES):
            if used[j]:
                # piece number j already used
                continue
                
            used[j] = 1
            
            for rot in range(3):
                # place the piece on the board
                board[i] = (j, rot)

                # stop the recursion if the current configuration
                # is not valid or a solution has been found
                if is_valid(board) and place(i + 1):
                    return True

                # remove the piece from the board
                board[i] = None
                
            used[j] = 0
            
        return False
    
    place(0)
    return board

In [40]:
res = solve_recursive()
print(res)
send_to_js('recursive', Permutation(res, []))

[(3, 2), (14, 1), (11, 2), (8, 2), (0, 2), (15, 1), (9, 0), (5, 2), (1, 2), (2, 1), (4, 1), (12, 2), (13, 0), (7, 0), (6, 1), (10, 1)]


<IPython.core.display.Javascript object>

In [41]:
%%javascript

drawStaticBoard('solution-recursive', 'recursive', element)

<IPython.core.display.Javascript object>

## Next Steps

- Create a p5.js [ipywiget](https://github.com/jupyter-widgets/ipywidgets)
- Better way to pass data from Javascript to Python (and reversed)